In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Normalization, Dense, Dropout, ReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard, ProgbarLogger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import os
import scipy.io as sio
import h5py
import sys
import importlib

use_gpu = 0

2025-04-07 15:50:13.226299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744033813.238596    2813 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744033813.242083    2813 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744033813.252292    2813 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744033813.252310    2813 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744033813.252312    2813 computation_placer.cc:177] computation placer alr

In [2]:
# ------------------ Configurazione GPU ------------------ #
if not use_gpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disabilita la GPU
print(f"Using GPU: {tf.config.list_physical_devices('GPU')}")

Using GPU: []


2025-04-07 15:50:16.927273: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## Load dataset

In [3]:

base_folder = '/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/'
input_folder = base_folder + 'Output Matlab/'

#DeepMIMO_dataset_folder = input_folder + 'DeepMIMO Dataset/'
DL_dataset_folder = input_folder + 'DL Dataset/'
network_folder_in = input_folder + 'Neural Network/'

output_folder = base_folder + 'Output Python/'
network_folder_out = output_folder + 'Neural Network/'
figure_folder = output_folder + 'Figures/'

seed = 0
np.random.seed(seed)

#simulate_data = 1

#if simulate_data == 1:
    # Simulated data for testing purposes
    #dataset_size = 100
    #training_size = 80 
    #validation_size = 20
    #M = 8
#else:
    #dataset_size = 36200
    #training_size = 30000
    #validation_size = 6200
    #M = 64*64 # 4096

#DL_input_reshaped = np.random.rand(M, 1, 1, dataset_size) 
#DL_output_reshaped = np.random.rand(1, 1, M, dataset_size)

My_ar = [32, 64]
Mz_ar = [32, 64]
Mx = 1
My = My_ar[0]
Mz = Mz_ar[0]

M_bar=8
Ur_rows = [1000, 1200]
#Training_Size=[2,  1e4*[1:.4:3]]
Training_Size = [30000]
Training_Size_dd = Training_Size[0]

#Import .mat files of datasets splits
filename_XTrain = DL_dataset_folder + 'XTrain' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'
filename_YTrain = DL_dataset_folder + 'YTrain' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'
filename_XValidation = DL_dataset_folder + 'XValidation' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'
filename_YValidation = DL_dataset_folder + 'YValidation' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'

print(filename_XTrain)
print(filename_YTrain)
print(filename_XValidation)
print(filename_YValidation)

/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Matlab/DL Dataset/XTrain_seed0_grid1200_M3232_Mbar8_30000.mat
/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Matlab/DL Dataset/YTrain_seed0_grid1200_M3232_Mbar8_30000.mat
/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Matlab/DL Dataset/XValidation_seed0_grid1200_M3232_Mbar8_30000.mat
/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Matlab/DL Dataset/YValidation_seed0_grid1200_M3232_Mbar8_30000.mat


In [4]:
# Load the data using h5py for MATLAB v7.3 files
with h5py.File(filename_XTrain, 'r') as f:
    X_train = f['XTrain'][:]
with h5py.File(filename_YTrain, 'r') as f:
    Y_train = f['YTrain'][:]
with h5py.File(filename_XValidation, 'r') as f:
    X_val = f['XValidation'][:]
with h5py.File(filename_YValidation, 'r') as f:
    Y_val = f['YValidation'][:]

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(30000, 1, 1, 1024)
(30000, 1024, 1, 1)
(6200, 1, 1, 1024)
(6200, 1024, 1, 1)


## Load Matlab trainedNet model

In [5]:
# Costruisci il path e il nome del modulo
modulename = 'trainedNet' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd)

# Aggiungi la cartella al path (quella che contiene la cartella del modello)
sys.path.append(network_folder_in)
filename_module = os.path.join(network_folder_in, modulename)

# Cambia la working directory temporaneamente
old_cwd = os.getcwd()
os.chdir(network_folder_in)

# Importa il modulo ed esegui il load
myModel = importlib.import_module(modulename)
#import model as myModel
model_mat = myModel.load_model()

# Ripristina la working directory originale
os.chdir(old_cwd)

# Ora puoi usare il modello
model_mat.summary()

print(model_mat.layers[1].mean.shape) # layers[1] is the Normalization layer
print(np.array(model_mat.layers[1].mean))

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_unnormalized (InputLayer) │ (None, 1024, 1, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ input_ (Normalization)          │ (None, 1024, 1, 1)     │         2,049 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully1_preFlatten1 (Reshape)    │ (None, 1, 1, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully1_ (Dense)                 │ (None, 1, 1, 1024)     │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 1, 1, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 1, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully2_preFlatten1 (Reshape)    │ (None, 1, 1, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully2_ (Dense)                 │ (None, 1, 1, 4096)     │     4,198,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully3_preFlatten1 (Reshape)    │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully3_ (Dense)                 │ (None, 1, 1, 4096)     │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully4_preFlatten1 (Reshape)    │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully4_ (Dense)                 │ (None, 1, 1, 1024)     │     4,195,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,226,689 (100.05 MB)

 Trainable params: 26,224,640 (100.04 MB)

 Non-trainable params: 2,049 (8.01 KB)

(1, 1024, 1, 1)
[[[[-5.1904644e-06]]

  [[-5.1904644e-06]]

  [[-5.1904644e-06]]

  ...

  [[-5.1904644e-06]]

  [[-5.1904644e-06]]

  [[-5.1904644e-06]]]]


In [13]:
# ------------------ DL Model Prediction ------------------ #
print("Start DL prediction for Figure 7...")

# Concatena XTrain + XValidation, cioè utilizza DL_input_reshaped
DL_input_reshaped = tf.concat([X_train, X_val], axis=0)
print(DL_input_reshaped.shape)  # (36200, 1, 1, 1024)

DL_input_reshaped = tf.transpose(DL_input_reshaped, perm=[0, 3, 1, 2]) 
print(DL_input_reshaped.shape)  # (36200, 1024, 1, 1)

# Esegui predizione con DL_input_reshaped
YPredictedFig7_mat = model_mat.predict(DL_input_reshaped)
print(YPredictedFig7_mat.shape)

# Recupera gli indici dei codebook
#Indmax_DL_mat = np.argmax(YPredictedFig7_mat, axis=1)
#print(Indmax_DL_mat.shape) # (36200, 1)

Start DL prediction for Figure 7...
(36200, 1, 1, 1024)
(36200, 1024, 1, 1)
   5/1132 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step

1132/1132 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step
(36200, 1024)


In [15]:
filename_YPredictedFig7_mat = network_folder_out + 'YPredictedFig7_mat' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '.mat'

# Scrittura in formato HDF5 (compatibile MATLAB v7.3)
with h5py.File(filename_YPredictedFig7_mat, 'w') as f:
    f.create_dataset('YPredictedFig7_mat', data=YPredictedFig7_mat)

print(f"Variabile salvata in {filename_YPredictedFig7_mat}")

Variabile salvata in /mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Python/Neural Network/YPredictedFig7_mat_seed0_grid1200_M3232_Mbar8.mat


## Recreate the same network in Python

In [ ]:
# Load normalization parameters
filename_trainedNet_scaler = network_folder_in + 'trainedNet_scaler' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'

with h5py.File(filename_trainedNet_scaler, 'r') as f:
    trainedNet_scaler = f['trainedNet_scaler'][:]

print(trainedNet_scaler.shape)
print(trainedNet_scaler) # should be -5.1904644e-06

In [ ]:
# Flatten the input and output arrays if necessary
#X = DL_input_reshaped.reshape(DL_input_reshaped.shape[0], -1).astype(np.float32)
#Y = DL_output_reshaped.reshape(DL_output_reshaped.shape[0], -1).astype(np.float32)

# Split the dataset into training and validation sets
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=validation_size / (training_size + validation_size), shuffle=False, random_state=seed)
#XTrain = np.array(DL_input_reshaped[:, 0, 0, Training_Ind], dtype=np.float32)
#YTrain = np.array(DL_output_reshaped[0, 0, :, Training_Ind], dtype=np.float32)
#XValidation = np.array(DL_input_reshaped[:, 0, 0, Validation_Ind], dtype=np.float32)
#YValidation = np.array(DL_output_reshaped[0, 0, :, Validation_Ind], dtype=np.float32)

In [ ]:
# Normalize the training data (zero-center normalization)
# Documentation StandardScaler: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
#  with_mean: default=True, If True, center the data before scaling. 
#  with_std: default=True, If True, scale the data to unit variance (or equivalently, unit standard deviation).
# Calcola la media e la deviazione standard per ogni feature (colonna) considerando tutti i campioni (lungo l'asse 0).
scaler = StandardScaler(with_std=False) # To match Matlab imageInputLayer normalization behavior

X_train_sq = np.squeeze(X_train)
X_val_sq = np.squeeze(X_val)

X_train_sq_scaled = scaler.fit_transform(X_train_sq)
print(f"Scaler: scale: {scaler.scale_}, mean: {scaler.mean_}, var: {scaler.var_}, n_features_in_: {scaler.n_features_in_}, n_samples_seen_: {scaler.n_samples_seen_}")

# Apply the same scaling to the validation data
X_val_sq_scaled = scaler.transform(X_val_sq)  # Transform the validation data using the same scaler

X_train_scaled = np.expand_dims(np.expand_dims(X_train_sq_scaled, axis=1), axis=2)
X_val_scaled = np.expand_dims(np.expand_dims(X_val_sq_scaled, axis=1), axis=2)
print(X_train_scaled.shape)
print(X_val_scaled.shape)


# Dovrebbe venire:                         -5.1904644e-06
print(np.mean(scaler.mean_))             # -5.190480343233745e-06
print(np.float32(np.mean(scaler.mean_))) # -5.1904804e-06



In [ ]:
# ------------------ DL Model Definition ------------------ #
# Define the neural network architecture
model_py = Sequential([
    Input(shape=(None, 1, 1, X_train.shape[3]), name='input'),

    # This layer will shift and scale inputs into a distribution centered around 0 with standard deviation 1. 
    # It accomplishes this by precomputing the mean and variance of the data, and calling (input - mean) / sqrt(var) at runtime.
    # The mean and variance values for the layer must be either supplied on construction or learned via adapt(). 
    # adapt() will compute the mean and variance of the data and store them as the layer's weights. 
    # adapt() should be called before fit(), evaluate(), or predict().
    Normalization(axis=3, mean=trainedNet_scaler, variance=1, name='normalization'),

    Dense(units=Y_train.shape[1], input_shape=(X_train.shape[1],), kernel_regularizer=l2(1e-4), name='Fully1'),
    ReLU(name='relu1'),
    Dropout(0.5, name='dropout1'),

    Dense(units=4 * Y_train.shape[1], kernel_regularizer=l2(1e-4), name='Fully2'),
    ReLU(name='relu2'),
    Dropout(0.5, name='dropout2'),

    Dense(units=4 * Y_train.shape[1], kernel_regularizer=l2(1e-4), name='Fully3'),
    ReLU(name='relu3'),
    Dropout(0.5, name='dropout3'),

    Dense(units=Y_train.shape[1], kernel_regularizer=l2(1e-4), name='Fully4'),
])

# Compile the model with SGD optimizer and mean squared error loss
optimizer = SGD(learning_rate=1e-1, momentum=0.9, nesterov=True)
model_py.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])
model_py.summary()

# ------------------ Learning Rate Scheduler ------------------ #
def lr_schedule(epoch, lr):
    if epoch > 0 and epoch % 3 == 0:
        return lr * 0.5  # Drop learning rate by factor of 0.5 every 3 epochs
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# ------------------ TensorBoard Callback ------------------ #
tensorboard_callback = TensorBoard(log_dir="./logs", histogram_freq=1)

# ------------------ Training Options ------------------ #
mini_batch_size = 500
verbose_frequency = max(1, Training_Size_dd // mini_batch_size)
progbar_logger = ProgbarLogger() #(count_mode='samples')

In [ ]:
# Save the trained model
model_py.save("trained_model.h5")
print("Model saved as 'trained_model.h5'")

# Save history and Y_predicted in .mat format to be imported in Matlab later
sio.savemat(output_folder + 'history.mat', {'history': history.history})
sio.savemat(output_folder + 'Y_predicted.mat', {'Y_predicted': Y_predicted})
print(f"Y_predicted saved as 'Y_predicted.mat' in {output_folder}")

#np.save(os.path.join(output_folder, 'history.npy'), history.history)
#np.save(os.path.join(output_folder, 'Y_predicted.npy'), Y_predicted)
#print("History and Y_predicted saved successfully.")

In [ ]:
# ------------------ DL Model Prediction ------------------ #
print("Start DL prediction for Figure 12...")
Y_predicted = model_py.predict(X_val)
print("Done")

print(f"Predicted output shape: {Y_predicted.shape}")